In [1]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")
print("Torch version:", torch.__version__)


CUDA available: True
GPU name: NVIDIA GeForce RTX 4070
Torch version: 2.5.1+cu121


In [2]:
# ======================================================
# Full Pipeline:
# 1. Multi-currency investment amount extraction (with unit + financial-report filter)
# 2. Location extraction
# 3. Climate-related keyword flag (E)
# 4. Duplicate detection (Sentence-BERT)
# Input:  global.csv with columns ["gvkey", "situation"]
# Output: 1118_with_amount_location_dup.csv
# ======================================================

import ssl
import chardet
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import re
import os

tqdm.pandas()

# Fix SSL in some Windows envs
ssl._create_default_https_context = ssl._create_unverified_context
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

# ======================================================
# PART 1: LOCATION DICTIONARIES + extract_location_code
# ======================================================

US_STATE_PATTERNS = {
    "AL": ["alabama", "birmingham", "montgomery", "mobile"],
    "AK": ["alaska", "anchorage"],
    "AZ": ["arizona", "phoenix", "tucson", "mesa"],
    "AR": ["arkansas", "little rock", "fayetteville"],
    "CA": ["california", "los angeles", "san francisco", "san diego", "sacramento", "san jose", "oakland"],
    "CO": ["colorado", "denver", "colorado springs", "aurora"],
    "CT": ["connecticut", "hartford", "new haven"],
    "DE": ["delaware", "wilmington", "dover"],
    "FL": ["florida", "miami", "orlando", "tampa", "jacksonville"],
    "GA": ["georgia", "atlanta", "savannah", "augusta"],
    "HI": ["hawaii", "honolulu"],
    "ID": ["idaho", "boise"],
    "IL": ["illinois", "chicago", "springfield", "aurora"],
    "IN": ["indiana", "indianapolis", "fort wayne"],
    "IA": ["iowa", "des moines"],
    "KS": ["kansas", "wichita", "topeka"],
    "KY": ["kentucky", "louisville", "lexington"],
    "LA": ["louisiana", "baton rouge", "new orleans", "lafayette"],
    "ME": ["maine", "portland", "augusta"],
    "MD": ["maryland", "baltimore", "rockville"],
    "MA": ["massachusetts", "boston", "cambridge", "worcester"],
    "MI": ["michigan", "detroit", "grand rapids"],
    "MN": ["minnesota", "minneapolis", "st. paul"],
    "MS": ["mississippi", "jackson"],
    "MO": ["missouri", "st. louis", "kansas city"],
    "MT": ["montana", "billings"],
    "NE": ["nebraska", "omaha", "lincoln"],
    "NV": ["nevada", "las vegas", "reno"],
    "NH": ["new hampshire", "manchester"],
    "NJ": ["new jersey", "newark", "jersey city"],
    "NM": ["new mexico", "albuquerque", "santa fe"],
    "NY": ["new york", "manhattan", "brooklyn", "queens", "buffalo", "rochester"],
    "NC": ["north carolina", "charlotte", "raleigh", "durham"],
    "ND": ["north dakota", "fargo"],
    "OH": ["ohio", "columbus", "cleveland", "cincinnati"],
    "OK": ["oklahoma", "oklahoma city", "tulsa"],
    "OR": ["oregon", "portland", "salem"],
    "PA": ["pennsylvania", "philadelphia", "pittsburgh"],
    "RI": ["rhode island", "providence"],
    "SC": ["south carolina", "charleston", "columbia"],
    "SD": ["south dakota", "sioux falls"],
    "TN": ["tennessee", "nashville", "memphis", "knoxville"],
    "TX": ["texas", "houston", "dallas", "austin", "san antonio", "el paso"],
    "UT": ["utah", "salt lake city", "provo"],
    "VT": ["vermont", "burlington"],
    "VA": ["virginia", "richmond", "virginia beach"],
    "WA": ["washington", "seattle", "tacoma"],
    "WV": ["west virginia", "charleston"],
    "WI": ["wisconsin", "milwaukee", "madison"],
    "WY": ["wyoming", "cheyenne"],
}

COUNTRY_PATTERNS = {
    
    "CAN": [" canada", " toronto", " vancouver", " lloydminster", " calgary", " montreal"],
    "MEX": [" mexico", " mexico city", " monterrey"],
    "BRA": [" brazil", " rio de janeiro", " sao paulo", " macae"],
    "ARG": [" argentina", " buenos aires"],
    "CHL": [" chile", " santiago"],
    "COL": [" colombia", " bogota"],
    "PER": [" peru", " lima"],

    "GBR": [" united kingdom", " uk ", " england", " london", " manchester", " belfast"],
    "FRA": [" france", " paris", " lyon"],
    "DEU": [" germany", " berlin", " munich", " frankfurt"],
    "ITA": [" italy", " rome", " milan"],
    "ESP": [" spain", " madrid", " barcelona"],
    "NLD": [" netherlands", " amsterdam"],
    "BEL": [" belgium", " brussels", " leuven", " louvain"],
    "CHE": [" switzerland", " zurich"],
    "SWE": [" sweden", " stockholm"],
    "NOR": [" norway", " oslo"],
    "DNK": [" denmark", " copenhagen"],
    "FIN": [" finland", " helsinki"],
    "IRL": [" ireland", " dublin"],
    "PRT": [" portugal", " lisbon"],

    "KOR": [" south korea", " korea ", " seoul", " yeosu"],
    "JPN": [" japan", " tokyo", " osaka"],
    "CHN": [" china", " beijing", " shanghai", " shenzhen", " guangzhou"],
    "HKG": [" hong kong"],
    "SGP": [" singapore"],
    "IND": [" india", " mumbai", " delhi"],
    "PAK": [" pakistan", " karachi"],
    "IDN": [" indonesia", " jakarta"],
    "PHL": [" philippines", " manila"],
    "VNM": [" vietnam", " hanoi", " ho chi minh"],

    "AUS": [" australia", " sydney", " melbourne"],
    "NZL": [" new zealand", " auckland"],

    "RUS": [" russia", " moscow", " saint petersburg"],
    "UKR": [" ukraine", " kyiv"],

    "SAU": [" saudi arabia", " dhahran", " riyadh"],
    "ARE": [" united arab emirates", " dubai", " abu dhabi"],
    "QAT": [" qatar", " doha"],
    "KWT": [" kuwait"],
    "OMN": [" oman", " muscat"],
    "BHR": [" bahrain"],

    "ZAF": [" south africa", " johannesburg", " cape town"],
    "EGY": [" egypt", " cairo"],
    "NGA": [" nigeria", " lagos"],
    "KEN": [" kenya", " nairobi"],
    "GHA": [" ghana", " accra"],
    "SEN": [" senegal", " taiba ndiaye"],

    "TUR": [" turkey", " istanbul"],
    "ISR": [" israel", " tel aviv"],

    "POL": [" poland", " warsaw"],
    "CZE": [" czech republic", " prague"],
    "AUT": [" austria", " vienna"],
    "HUN": [" hungary", " budapest"],
    "GRC": [" greece", " athens"],
    "ROU": [" romania", " bucharest"],

    "THA": [" thailand", " bangkok"],
    "MYS": [" malaysia", " kuala lumpur"],
    "KAZ": [" kazakhstan", " almaty"],
    "UZB": [" uzbekistan"],
}

def extract_location_code(text: str):
    """
    Return a 2-letter US state code or 3-letter country code
    based on simple keyword matching in the text.
    If none found, return None.
    """
    if not isinstance(text, str):
        return None
    t = " " + text.lower() + " "

    # First match US states
    for code, patterns in US_STATE_PATTERNS.items():
        for p in patterns:
            if p in t:
                return code

    # Then match countries
    for code, patterns in COUNTRY_PATTERNS.items():
        for p in patterns:
            if p in t:
                return code

    return None

# ======================================================
# PART 2: INVESTMENT AMOUNT + UNIT + FINANCIAL REPORT FILTER
# ======================================================

INVEST_KEYWORDS = [
    # 1) Direct investment / capital
    "invest", "investment", "invests", "invested", "investing",
    "capital expenditure", "capex", "fund", "funding", "financing",
    "finance", "raise capital", "joint venture", "jv",
    "partnership", "strategic partnership",

    # 2) Expansion / new projects
    "expand", "expansion", "expanding",
    "construct", "construction", "build", "building", "built",
    "upgrade", "upgrading", "renovate",
    "launch project", "project launch",
    "project start", "project begins", "groundbreaking",

    # 3) Factory / production / capacity
    "factory", "plant", "facility", "manufacturing plant",
    "production", "production line",
    "capacity", "increase capacity", "capacity expansion",
    "assembly plant", "manufacturing capacity",
    "processing plant", "enrichment plant",
    "smelter", "mill", "refinery", "mine", "mining project",

    # 4) R&D / technology centers
    "research center", "r&d center", "innovation center",
    "development center", "engineering center",
    "lab", "laboratory", "tech hub",

    # 5) Real estate / large facilities
    "office building", "campus", "headquarters", "hq",
    "shopping center", "mall", "hotel", "resort", "casino",
    "casino resort", "data center", "distribution center",
    "logistics center", "warehouse", "fulfillment center",
    "infrastructure", "construction project",

    # 6) New stores / branches
    "open store", "opening", "grand opening",
    "new location", "new branch", "retail location",
    "store opening", "store launch",

    # 7) M&A / equity investments
    "acquire", "acquisition", "buy", "merger", "purchase",
    "takeover", "equity stake", "buyout", "minority stake",
    "majority stake",

    # 8) Energy / resource projects
    "hydrogen plant", "power plant", "solar farm", "wind farm",
    "lithium project", "battery plant", "ev plant",
    "pipeline", "refinery expansion",
    "smelting capacity", "mine expansion",

    # 9) Banking / financial branches
    "branch opening", "bank branch",
    "opening office", "new office", "regional office",
]

# Financial-report keyword list
FINANCIAL_REPORT_KEYWORDS = [
    "financial results",
    "results for the", "results for the first quarter",
    "results for the second quarter", "results for the third quarter",
    "results for the fourth quarter",
    "quarter ended", "for the quarter ended",
    "for the year ended", "fiscal year", "fiscal 20",
    "net income", "net loss",
    "earnings per share", "eps",
    "profit", "profits",
    "income", "operating income", "operating loss",
    "revenue", "revenues", "net sales", "sales were",
    "gross margin", "operating margin",
    "ebitda", "adjusted ebitda",
    "cash flow", "cash flows from operations",
    "balance sheet", "statement of operations",
    "consolidated financial statements",
    "dividend", "dividends per share",
]

# Multi-currency symbols
CURRENCY_SYMBOLS = "$€¥￥£"

# Range like "$1,000 to $2,000"
RANGE_PATTERN = re.compile(
    rf"([{CURRENCY_SYMBOLS}])\s*(\d[\d,]*)\s*(to|-)\s*\1\s*(\d[\d,]*)",
    re.IGNORECASE
)

# Single amount with optional unit after the number: $100 million, €2.5bn, ¥300k, etc.
CURRENCY_PATTERN = re.compile(
    rf"([{CURRENCY_SYMBOLS}])\s*(\d[\d,]*(?:\.\d+)?)\s*"
    r"(million|billion|thousand|m|bn|k|crore|lakh)?",
    re.IGNORECASE
)

def simple_sent_tokenize(text: str):
    """Very simple sentence splitter based on punctuation."""
    if not isinstance(text, str):
        return []
    text = text.strip()
    if not text:
        return []
    parts = re.split(r'(?<=[.!?])\s+', text)
    return [p.strip() for p in parts if p.strip()]

def clean_num(s: str) -> float:
    """Remove commas and convert to float."""
    return float(s.replace(",", ""))

def is_financial_report(text: str) -> bool:
    """
    Heuristic: if the text looks like an earnings/financial report
    (multiple mentions of financial-report-related phrases),
    we treat the whole text as a non-investment context.
    """
    if not isinstance(text, str) or not text.strip():
        return False
    t = text.lower()
    hits = 0
    for kw in FINANCIAL_REPORT_KEYWORDS:
        if kw in t:
            hits += 1
            if hits >= 2:
                return True
    return False

def extract_amount_from_sentence(sent: str):
    """
    Extract amount and unit from a single sentence:
    - Only consider sentences containing at least one INVEST_KEYWORD.
    - Look for currency amounts (with symbols).
    - If a range like "$1000 to $2000" is found, return (range_string, "").
    - Otherwise, sum all numbers after currency symbols in this sentence
      and infer a common unit if possible.
    Returns:
        None         -> no valid investment amount
        (value, unit)
            - value: float (summed) OR str (for range)
            - unit:  unit string like 'million', 'bn', 'k', etc., or "" if none.
    """
    s_lower = sent.lower()

    # Must look like an investment-related sentence
    if not any(kw in s_lower for kw in INVEST_KEYWORDS):
        return None

    # First, check for explicit range "$1000 to $2000"
    rng_match = RANGE_PATTERN.search(sent)
    if rng_match:
        # Keep the full range string as requested, unit left blank
        return rng_match.group(0), ""

    # Otherwise, collect all currency amounts in this sentence
    matches = CURRENCY_PATTERN.findall(sent)
    if not matches:
        return None

    total = 0.0
    units = []

    for symbol, num_str, unit in matches:
        total += clean_num(num_str)
        if unit:
            units.append(unit.lower())

    # Decide the unit for this sentence:
    # if all non-empty units are the same, keep it; else, use ""
    sentence_unit = ""
    if units:
        unique_units = set(units)
        if len(unique_units) == 1:
            sentence_unit = unique_units.pop()

    return total, sentence_unit

def extract_amount_and_unit(text: str):
    """
    Main entry for amount + unit extraction from a full text:
    1) If it's a financial report → return (None, "").
    2) Else, split into sentences and run sentence-level extraction.
       - Keep the largest numeric amount across sentences.
       - Keep the first range string if any (and prefer range over numeric).
    Returns:
        (value, unit)
        - value: float or str (for range), or None
        - unit:  unit string or "" if no unit
    """
    if not isinstance(text, str):
        return None, ""

    if is_financial_report(text):
        # Financial report → ignore as investment
        return None, ""

    sentences = simple_sent_tokenize(text)

    best_numeric = None
    best_numeric_unit = ""

    best_range = None
    best_range_unit = ""

    for sent in sentences:
        res = extract_amount_from_sentence(sent)
        if res is None:
            continue

        val, unit = res

        # Range case: keep the first range encountered
        if isinstance(val, str):
            if best_range is None:
                best_range = val
                best_range_unit = unit or ""
        else:
            # Numeric case: keep the largest numeric value
            if best_numeric is None or val > best_numeric:
                best_numeric = val
                best_numeric_unit = unit or ""

    # If there is any range, prefer it
    if best_range is not None:
        return best_range, best_range_unit

    return best_numeric, best_numeric_unit

# ======================================================
# PART 2.5: CLIMATE / ENVIRONMENT KEYWORDS  ->  E flag
# ======================================================

CLIMATE_KEYWORDS_RAW = """
renewable energy,electric vehicle,clean energy,new energy,climate,energy,natural gas,hydrogen,natural disaster,
wind power,wind energy,energy efficient,greenhouse gas,solar energy,
air quality,clean air,carbon emission,gas emission,extreme weather,emission,carbon,
carbon dioxide,water resource,autonomous vehicle,energy environment,
wind resource,government india,battery power,air pollution,battery electric,
integrate resource,clean power,carbon price,world population,solar farm,
energy regulatory,obama administration,heat power,carbon tax,unite nation,
onshore wind,electric motor,provide energy,efficient solution,global warm,
power generator,solar pv,scale solar,need clean,coastal area,energy star,
environmental footprint,design use,area energy,charge station,clean water,
major design,vehicle manufacturer,motor control,combine heat,electric bus,
distribute power,environmental benefit,eco friendly,electrical vehicle,
carbon neutral,fast charge,cell power,energy team,cycle gas,
coal gasification,environmental concern,carbon intensity,energy application,
produce electricity,help state,environmental standard,power agreement,
supply energy,electric hybrid,source power,sustainability goal,energy reform,
plant power,compare conventional,gas vehicle,effort energy,pass house,
carbon free,driver assistance,electrical energy,solar installation,snow ice,
renewable natural,promote use,farm project,laser diode,deliver energy,
protect environment,sustainable energy,manage energy,invest energy,
electric energy,forest land,capacity energy,unite nation,
hurricane,hurricanes,storms,drought,flooding,flood,wildfire,wildfires,
windstorm,storm losses,severe winter,storm related,storm activity,polar vortex,
storm season,storm damage,droughts,tropical storm,snowstorms,winter storm,
hailstorm,volcano,extreme cold,cold winter,heat wave,heating season,
precipitation,ice season,snowfall,rainfall,degree days,winter conditions,
warm winter,heavy rains,cold summer,unseasonably warm,harsh winter,clouds,
the warmest,early winter,the warm,cool summer,cold wind,heavy winter,
the rain,the winds,energy efficiency,alternative energy,superior energy,higher energy,
renewable,ecosystem,energy management,the carbon,green energy,
shale gas,fuel efficient,solar power,battery,wind farm,
fuel economy,solar cell,energy future,solar projects,decarbonization,
emissions,climate risk,climate change,biodiversity,low carbon,

decarbon,decarbonize,decarbonized,decarbonization,
net zero,net-zero,carbon neutrality,
emission reduce,emission reduction,reduce emission,lower emission,
low-carbon,carbon footprint,GHG emission,greenhouse gas emission,
methane leak,methane emission,CH4 emission,
climate-related risk,climate resilience,resilience plan,
physical risk,transition risk,environmental risk,sustainability risk,ESG risk,
environmental sustainability,corporate sustainability,sustainable finance,
energy transition,transition energy,transition plan,green transition,
renewable project,renewable capacity,renewable expansion,
clean technology,cleantech,green technology,sustainable technology,
green innovation,eco innovation,eco-friendly,environmentally friendly,
biofuel,biodiesel,biogas,bioenergy,
hydrogen energy,green hydrogen,blue hydrogen,H2 production,hydrogen project,
carbon capture,CCS,carbon capture storage,CCUS,carbon sequestration,
direct air capture,DAC,air capture technology,
energy storage,battery storage,grid storage,
sea level rise,coastal flooding,coastal erosion,
environmental regulation,EPA regulation,emission regulation,
climate regulation,carbon regulation,emission standard,
carbon pricing,carbon market,emission trading,cap and trade,
pollution control,water pollution,soil contamination,
environmental impact,environmental compliance,environmental penalty,
sustainable material,recycled material,eco material,
green building,LEED certified,energy-efficient building,
energy conservation,efficiency upgrade,
solar panel,solar capacity,solar generation,
wind turbine,wind installation,wind capacity,
geothermal energy,tidal energy,wave energy,
EV charging,charging infrastructure,EV infrastructure,
electric mobility,clean mobility,zero emission vehicle,
sustainable supply chain,green supply chain,low-carbon supply chain,
circular economy,recycling program,waste reduction,waste management,
nature-based solution,forest conservation,reforestation,afforestation,
ecosystem service,habitat restoration,
paris agreement,paris climate agreement,climate summit,
science based target,science-based target,

GHG protocol,greenhouse inventory,emission inventory,
carbon offset,carbon offsets,carbon credit,carbon credits,carbon trading,
carbon removal,carbon removals,negative emission,negative emissions,
REDD+,avoided deforestation,deforestation free,zero deforestation,
climate mitigation,mitigation measure,mitigation strategy,
climate adaptation,adaptation measure,adaptation strategy,
climate scenario,two degree scenario,temperature pathway,
low-carbon transition,net-zero transition,transition pathway,
resilient infrastructure,climate-resilient,weather-resilient,
hardening infrastructure,grid hardening,storm hardening,
flood defence,flood wall,sea wall,levee system,
microgrid,smart grid,grid modernization,grid upgrade,grid resilience,
demand response,peak shaving,load shifting,flexible load,
heat pump,heat pumps,district heating,district cooling,
building retrofit,deep retrofit,energy retrofit,
high-efficiency appliance,efficient appliance,high efficiency motor,
vehicle electrification,transport electrification,fleet electrification,
zero-emission truck,zero-emission bus,electric fleet,
green logistics,low-carbon logistics,clean logistics,
sustainable aviation fuel,low carbon fuel standard,
fuel switching,coal-to-gas,coal phase-out,coal phase down,
stranded asset,carbon-intensive asset,high-carbon asset,
climate disclosure,TCFD,climate reporting,climate-related disclosure,
ESG report,sustainability report,non-financial reporting,
green bond,climate bond,sustainability-linked bond,SLB,
green loan,sustainability-linked loan,
impact investing,climate finance,green finance,
nature risk,nature-related risk,TNFD,
natural capital,ecosystem capital,ecosystem resilience,
land use change,land-use change,LULUCF,
peatland,mangrove,wetland restoration,
water stress,water scarcity,water shortage,water risk,
drought resistant,drought tolerant,water saving technology,
storm surge,coastal storm,tidal surge,
urban heat island,heat island effect,
air quality index,AQI,fine particulate,PM2.5,PM10,
nitrogen oxide,NOx emission,sulfur dioxide,SO2 emission,ozone pollution,smog episode,
waste heat recovery,heat recovery system,
life cycle assessment,life-cycle assessment,LCA study,
cradle to grave,cradle-to-cradle,
eco design,ecodesign,design for environment,
hazardous waste,toxic release,chemical spill,
sustainable agriculture,climate smart agriculture,regenerative agriculture,
organic farming,soil carbon,soil sequestration,
reforestation project,tree planting program,tree plantation,
water stewardship,responsible water use,
green procurement,sustainable procurement,responsible sourcing,
low-impact material,low impact material,
biodegradable material,compostable material,
ocean plastic,plastic waste reduction,single-use plastic phase-out,
zero waste,landfill diversion,waste diversion,
renewable portfolio standard,RPS target,
feed-in tariff,renewable incentive,green tariff,
time-of-use pricing,dynamic pricing,
vehicle-to-grid,V2G,
energy benchmark,efficiency benchmark,
climate lawsuit,climate litigation,
environmental justice,climate justice,
just transition,worker transition plan,
eco-label,environmental label,energy label
"""

def build_climate_keywords(raw: str):
    tokens = re.split(r'[,\|，]', raw)
    return sorted({t.strip().lower() for t in tokens if t.strip()})

CLIMATE_KEYWORDS = build_climate_keywords(CLIMATE_KEYWORDS_RAW)

def has_climate_keyword(text: str) -> int:
    """
   
    """
    if not isinstance(text, str) or not text.strip():
        return 0
    t = text.lower()
    for kw in CLIMATE_KEYWORDS:
        if kw in t:
            return 1
    return 0

# ======================================================
# PART 3: LOAD CSV
# ======================================================

events_file_path = "C:/Users/xj151/OneDrive/Desktop/global.csv"

with open(events_file_path, "rb") as f:
    enc = chardet.detect(f.read())["encoding"]
print("Detected encoding:", enc)

df = pd.read_csv(events_file_path, encoding=enc, on_bad_lines="skip", engine="python")
df = df.copy().reset_index(drop=True)

required_cols = ["gvkey", "situation"]
for col in required_cols:
    if col not in df.columns:
        raise ValueError(f"Column '{col}' not found in input CSV")

df["situation"] = df["situation"].fillna("")
df["gvkey"] = df["gvkey"].fillna("UNKNOWN")

print("Total rows in file:", len(df))
print(df[["gvkey", "situation"]].head())

# ======================================================
# PART 4: LOCATION + AMOUNT+UNIT + CLIMATE FLAG (E)
# ======================================================

print("Extracting location codes for all rows...")
df["location"] = df["situation"].progress_apply(extract_location_code)

print("Extracting investment amounts + units for all rows...")
df[["invest_amount_value", "invest_amount_unit"]] = df["situation"].progress_apply(
    lambda x: pd.Series(extract_amount_and_unit(x))
)

print("Flagging climate-related events (E)...")
df["E"] = df["situation"].progress_apply(has_climate_keyword)

print(df[["gvkey", "location", "invest_amount_value", "invest_amount_unit", "E"]].head(20))

# ======================================================
# PART 5: Sentence-BERT Duplicate Detection
# ======================================================

model_name = "sentence-transformers/all-mpnet-base-v2"

print("Loading Sentence-BERT tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name)

print("Loading Sentence-BERT model...")
model = AutoModel.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

print("Model loaded on device:", device)

def get_sentence_embedding(text: str) -> np.ndarray:
    """
    Compute a sentence embedding using:
    - Transformer last hidden states
    - Attention-mask-based mean pooling (ignore padding)
    - L2 normalization
    """
    if not isinstance(text, str) or text.strip() == "":
        return np.zeros(model.config.hidden_size, dtype=np.float32)

    enc = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=256,
        padding=True
    )
    enc = {k: v.to(device) for k, v in enc.items()}

    with torch.no_grad():
        out = model(**enc)

    last_hidden = out.last_hidden_state         # (1, seq_len, hidden)
    mask = enc["attention_mask"].unsqueeze(-1)  # (1, seq_len, 1)

    masked = last_hidden * mask
    summed = masked.sum(dim=1)                  # (1, hidden)
    lengths = mask.sum(dim=1)                   # (1, 1)
    emb = (summed / lengths).squeeze(0).cpu().numpy()

    norm = np.linalg.norm(emb)
    if norm > 0:
        emb = emb / norm
    return emb.astype(np.float32)

print("Computing embeddings for all situations...")
emb_list = df["situation"].progress_apply(get_sentence_embedding).values
emb_matrix = np.vstack(emb_list)
print("Embedding matrix shape:", emb_matrix.shape)

# ======================================================
# PART 6: DUPLICATE FLAG (dup) — group by gvkey
# ======================================================

SIM_THRESHOLD = 0.9
dup_flags = np.zeros(len(df), dtype=int)

for gv, g in df.groupby("gvkey"):
    idx_list = list(g.index)
    rep_indices = []

    for i in idx_list:
        loc_i = df.loc[i, "location"]
        emb_i = emb_matrix[i].reshape(1, -1)

        if not rep_indices:
            dup_flags[i] = 0
            rep_indices.append(i)
            continue

        candidate_indices = []
        for j in rep_indices:
            loc_j = df.loc[j, "location"]
            if (pd.notna(loc_i)) and (pd.notna(loc_j)) and (loc_i != loc_j):
                continue
            candidate_indices.append(j)

        if not candidate_indices:
            dup_flags[i] = 0
            rep_indices.append(i)
            continue

        rep_embs = emb_matrix[candidate_indices]
        sims = cosine_similarity(emb_i, rep_embs)[0]
        max_sim = sims.max()

        if max_sim >= SIM_THRESHOLD:
            dup_flags[i] = 1
        else:
            dup_flags[i] = 0
            rep_indices.append(i)

df["dup"] = dup_flags

print("Duplicate flag counts (all rows):")
print(df["dup"].value_counts())
print(df[["gvkey", "location", "invest_amount_value", "invest_amount_unit", "E", "dup"]].head(30))

# ======================================================
# SAVE RESULT
# ======================================================

output_path = "C:/Users/xj151/OneDrive/Desktop/1118_with_amount_location_dup.csv"
df.to_csv(output_path, index=False, encoding="utf-8")
print("Saved:", output_path)


Detected encoding: MacRoman
Total rows in file: 148847
   gvkey                                          situation
0  64536  Euronet Worldwide Inc. has opened a new data p...
1  16643  AMCORE Financial Inc. announced the opening of...
2  16784  Commerce Bancorp Inc. (NJ) said that it has es...
3  23671  Nokia Corp. boasting a fashionable new design ...
4  16926  Mayflower Co-operative Bank announced that it ...
Extracting location codes for all rows...


100%|████████████████████████████████████████████████████████████████████████| 148847/148847 [00:15<00:00, 9468.16it/s]


Extracting investment amounts + units for all rows...


100%|████████████████████████████████████████████████████████████████████████| 148847/148847 [00:28<00:00, 5311.97it/s]


Flagging climate-related events (E)...


100%|████████████████████████████████████████████████████████████████████████| 148847/148847 [00:28<00:00, 5227.14it/s]


     gvkey location invest_amount_value invest_amount_unit  E
0    64536      HUN                None                     1
1    16643       IL                None                     0
2    16784       NY                None                     0
3    23671       AL                None                     0
4    16926       MA                None                     0
5    16243       IL                None                     0
6    25896       AL                None                     0
7     2136       DE                None                     0
8    10581      BRA                None                     0
9    25950       FL                None                     0
10    8446       OH                None                     0
11  226659      JPN                None                     0
12   29901       IL                None                     0
13    7139     None                None                     0
14  122062      CHN                None                     0
15   639

100%|██████████████████████████████████████████████████████████████████████████| 148847/148847 [41:02<00:00, 60.46it/s]


Embedding matrix shape: (148847, 768)
Duplicate flag counts (all rows):
dup
0    134692
1     14155
Name: count, dtype: int64
     gvkey location invest_amount_value invest_amount_unit  E  dup
0    64536      HUN                None                     1    0
1    16643       IL                None                     0    0
2    16784       NY                None                     0    0
3    23671       AL                None                     0    0
4    16926       MA                None                     0    0
5    16243       IL                None                     0    0
6    25896       AL                None                     0    0
7     2136       DE                None                     0    0
8    10581      BRA                None                     0    0
9    25950       FL                None                     0    0
10    8446       OH                None                     0    0
11  226659      JPN                None                     0    0
12 